## This verifies that there is the same amount of labels and images and they match

In [1]:
import os
from ml_carbucks import DATA_DIR

std_dir = DATA_DIR / "final_carbucks" / "standard"
splits = ["train", "val", "test"]
for split in splits:
    for file in os.listdir(os.path.join(std_dir, "labels", split)):
        key = file.split(".txt")[0]
        
        image_exists = os.path.exists(os.path.join(std_dir, "images", split, f"{key}.jpg"))

        if not image_exists:
            print(f"Image {split}/{key} does not exist: path {os.path.join(std_dir, 'labels', split, f'{key}.txt')}")

for split in splits:
    for file in os.listdir(os.path.join(std_dir, "images", split)):
        key = file.split(".jpg")[0]
        
        label_exists = os.path.exists(os.path.join(std_dir, "labels", split, f"{key}.txt"))

        if not label_exists:
            print(f"Label {split}/{key} does not exist: path {os.path.join(std_dir, 'images', split, f'{key}.jpg')}")

## This filters the dataset by category only

In [2]:
import os
import random
from pathlib import Path
from pprint import pprint as print

import yaml

from ml_carbucks.utils.conversions import convert_yolo_to_coco
from ml_carbucks.utils.logger import setup_logger

logger = setup_logger(__name__)


def filter_dataset(data_dir: str | Path, splits:list[str], class_to_keep:str="2", negative_sample_ratio:float=-1):

    dataset_yaml_path = os.path.join(data_dir, "dataset_single.yaml")

    if not os.path.exists(dataset_yaml_path):
        logger.info(f"Dataset yaml file does not exist. Proceeding...")
    else:
        loaded_yaml = None
        with open(dataset_yaml_path, "r") as f:
            loaded_yaml = yaml.safe_load(f)
        if loaded_yaml['nc'] == 1 and loaded_yaml['names'] == [f'class_to_keep_{class_to_keep}']:
            logger.info(f"Dataset already filtered to class {class_to_keep}. Exiting...")
            return


    for split in splits:
        obj = {}
        for file in os.listdir(os.path.join(data_dir, "labels", split)):
            if file.endswith(".txt"):
                with open(os.path.join(data_dir, "labels", split, file), "r") as f:
                    lines = f.readlines()
                    file_classes = set()
                    for line in lines:
                        class_id = line.split()[0]
                        file_classes.add(class_id)
                    obj[file.split(".")[0]] = list(file_classes)

        files_to_keep = []
        for key, value in obj.items():
            if class_to_keep in value:
                files_to_keep.append(key)

        files_neg_to_keep = []
        for key, value in obj.items():
            if class_to_keep not in value:
                if negative_sample_ratio < 0:
                    files_neg_to_keep.append(key)
                elif len(files_neg_to_keep) < len(files_to_keep) and random.random() < negative_sample_ratio:
                    files_neg_to_keep.append(key)
                else:
                    pass
                
        
        print(f"{len(files_to_keep)}, {len(files_neg_to_keep)}")

        for key, value in obj.items():
            if key in files_to_keep or key in files_neg_to_keep:
                # clean up other classes from the labels
                with open(os.path.join(data_dir, "labels", split, f"{key}.txt"), "r") as f:
                    lines = f.readlines()
                with open(os.path.join(data_dir, "labels", split, f"{key}.txt"), "w") as f:
                    for line in lines:
                        class_id = line.split()[0]
                        if class_id == class_to_keep:
                            f.write(line.replace(f"{class_to_keep} ", "0 "))
            else:
                os.remove(os.path.join(data_dir, "labels", split, f"{key}.txt"))
                os.remove(os.path.join(data_dir, "images", split, f"{key}.jpg"))   
    
    # write dataset.yaml file
    dataset_yaml = {
        'nc': 1,
        'names': [f'class_to_keep_{class_to_keep}']
    }

    for split in splits:
        dataset_yaml[split] = f"images/{split}"

    with open(dataset_yaml_path, "w") as f:
        yaml.dump(dataset_yaml, f)

    convert_yolo_to_coco(data_dir, splits)

    

## Test it out

In [ ]:
from ml_carbucks import DATA_DIR, RESULTS_DIR

data_dir = DATA_DIR / "final_carbucks" / "crack"
splits = ["train", "val", "test"]
class_to_keep = "2"  # crack class
filter_dataset(data_dir, splits, class_to_keep=class_to_keep, negative_sample_ratio=-1)



INFO __main__ 12:11:24 | Dataset yaml file does not exist. Proceeding...
'332, 2940'
'92, 708'
'27, 273'
INFO ml_carbucks.utils.conversions 12:11:25 | Saved /home/bachelor/ml-carbucks/data/final_carbucks/crack/instances_train_curated.json with 3272 images and 430 annotations.
INFO ml_carbucks.utils.conversions 12:11:25 | Saved /home/bachelor/ml-carbucks/data/final_carbucks/crack/instances_val_curated.json with 800 images and 107 annotations.
INFO ml_carbucks.utils.conversions 12:11:25 | Saved /home/bachelor/ml-carbucks/data/final_carbucks/crack/instances_test_curated.json with 300 images and 37 annotations.


In [4]:
from ultralytics.models.yolo import YOLO

model = YOLO("yolo11l.pt")

model.train(
    data="/home/bachelor/ml-carbucks/data/final_carbucks/crack/dataset_single.yaml",
    epochs=50,
    batch=16,
    imgsz=640,
    project=RESULTS_DIR / "experiments" / "yolo11l_crack"
)

New https://pypi.org/project/ultralytics/8.3.233 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.229 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA RTX 4000 SFF Ada Generation, 20154MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/bachelor/ml-carbucks/data/final_carbucks/crack/dataset_single.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l.pt, momentum=0.937, mos

KeyboardInterrupt: 

## Results

After running it by python files, it seems that the best result was at around 0.8 which is pretty good but not much better than with all categories

It could be tested again with more epochs or different parameters